# ReActNet
This notebook demonstrates the implementation of the ReActNet Paper
# Steps to binarize the model
- Load Dataset and DataLoader
- Create the teacher model to be binarized
- Load the configuration YAML file
- create `ReActNet object` and pass the teacher model, dataloader, configuration
- Binarize the model by using `compress_model` method

In [9]:
import sys
import os
import shutil

sys.path.append("../../../")
sys.path.append("../../../../../../")

import torch
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
from trailmet.algorithms.binarize.utils import *
from trailmet.algorithms.binarize.ReActNet import ReActNet
from trailmet.models.models import models
import torchvision

import yaml

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [11]:
root_dir = "./"
import os

with open(os.path.join(root_dir, "reactnet_cifar100.yaml"), "r") as stream:
    kwargs = yaml.safe_load(stream)
dataset_args = kwargs["GENERAL"]
kwargs

{'GENERAL': {'DATASET': 'c100',
  'BACKBONE': 'mobilenetv2',
  'batch_size': 256,
  'workers': 4,
  'valid_size': 0.1,
  'num_train': 0,
  'num_classes': 100,
  'insize': 32},
 'ReActNet1_ARGS': {'batch_size': 256,
  'epochs': 5,
  'learning_rate': 0.0005,
  'momentum': 0.9,
  'weight_deacy': '1e-5',
  'label_smooth': 0.1,
  'save': './save_path_mbnet_cifar100_0fp/step1'},
 'ReActNet2_ARGS': {'batch_size': 256,
  'epochs': 5,
  'learning_rate': 0.0005,
  'momentum': 0.9,
  'weight_deacy': 0,
  'label_smooth': 0.1,
  'save': './save_path_mbnet_cifar100_0fp/step2'}}

In [12]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print("> SEEDING DONE")


set_seed(1024)

> SEEDING DONE


# Defining the Model 

In [16]:
model_name = "mobilenetv2"
teacher = models.get_model("mobilenetv2", "full", 100, 32)

pret = torch.load(
    "/workspace/code/janhavi/ChipNet/weights/mobilenetv2_c100_pretrained.pth",
    map_location="cuda",
)
pret = pret["state_dict"]

teacher.load_state_dict(pret)

# Augmentation, Dataset and DataLoaders

In [17]:
# Augmentation
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
        Lighting(lighting_param),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

valid_transform = transforms.Compose(
    [transforms.Resize(36), transforms.CenterCrop(32), transforms.ToTensor()]
)
test_transform = valid_transform

input_transforms = {
    "train": train_transform,
    "val": valid_transform,
    "test": test_transform,
}

target_transforms = {"train": None, "val": None, "test": None}

# Create Dataset
cifar100_dataset = DatasetFactory.create_dataset(
    name="CIFAR100",
    root=root_dir,
    split_types=["train", "val", "test"],
    val_fraction=0.15,
    transform=input_transforms,
    target_transform=target_transforms,
)
# Define DataLoaders
train_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["train"],
    batch_size=dataset_args["batch_size"],
    sampler=cifar100_dataset["train_sampler"],
    num_workers=dataset_args["workers"],
)
val_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["val"],
    batch_size=dataset_args["batch_size"],
    sampler=cifar100_dataset["val_sampler"],
    num_workers=dataset_args["workers"],
)
test_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["test"],
    batch_size=dataset_args["batch_size"],
    sampler=cifar100_dataset["test_sampler"],
    num_workers=dataset_args["workers"],
)

dataloaders = {"train": train_loader100, "val": val_loader100, "test": test_loader100}

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# ReActNet Object

In [18]:
a = ReActNet(teacher, model_name, dataloaders, num_fp=0, **kwargs)

# Binarizing the model

In [19]:
fin = a.compress_model()

Step-1 Training with activations binarized for 5 epochs

EPOCH-0



100%|███████████████████████████████████| 30/30 [00:01<00:00, 18.63it/s, acc1=0.99, acc5=4.76, loss=4.61]

Average Train Loss-4.607090417704863
Top-1 Train Accuracy-1.0141176470588236
Top-5 Train Accuracy-4.863529411764706
Validation Accuracy-0.9895833333333334
Validation Loss-4.605341053009033
EPOCH-1




100%|███████████████████████████████████| 30/30 [00:01<00:00, 17.82it/s, acc1=0.981, acc5=5.6, loss=4.61]

Average Train Loss-4.605135306952981
Top-1 Train Accuracy-0.9129411764705883
Top-5 Train Accuracy-4.83764705882353
Validation Accuracy-0.9813596487045289
Validation Loss-4.605096832911173
EPOCH-2



...

Average Train Loss-4.605048231326832
Top-1 Train Accuracy-0.9435294117647058
Top-5 Train Accuracy-4.903529411764706
Validation Accuracy-0.8723958333333334
Validation Loss-4.6055182933807375
EPOCH-3




100%|██████████████████████████████████| 30/30 [00:02<00:00, 11.11it/s, acc1=0.924, acc5=4.87, loss=4.61]

Average Train Loss-4.605046349828384
Top-1 Train Accuracy-1.0070588235294118
Top-5 Train Accuracy-5.0329411764705885
Validation Accuracy-0.9244791666666666
Validation Loss-4.605253887176514
EPOCH-4




100%|██████████████████████████████████| 30/30 [00:02<00:00, 11.62it/s, acc1=0.955, acc5=4.94, loss=4.61]

Average Train Loss-4.605047231741512
Top-1 Train Accuracy-1.011764705882353
Top-5 Train Accuracy-5.181176470588236
Validation Accuracy-0.9553179820378621
Validation Loss-4.605306879679362


# Evaluate binarized model on test set

In [ ]:
chk = torch.load("./save_path_mbnet_cifar100_0fp/step2/c100-model_best.pth.tar")

In [ ]:
chk["best_top1_acc"]